In [103]:
# This code downloads current weather data and forecasts from https://openweathermap.org/forecast5 for St Albans
# My API key: 4d9e86afd3e0034d2ad0b63e6c30da28
# current weather API is explained here: https://openweathermap.org/current
# forecast weather API is explained here: https://openweathermap.org/forecast5

# current weather by city name:   api.openweathermap.org/data/2.5/weather?q=St Albans&units=metric&limit=1&appid=4d9e86afd3e0034d2ad0b63e6c30da28
# current weather by lat & long:   api.openweathermap.org/data/2.5/weather?lat=51.7564&lon=-0.3519&units=metric&limit=1&appid=4d9e86afd3e0034d2ad0b63e6c30da28
# forecast weather by city name:   api.openweathermap.org/data/2.5/forecast?q=St Albans&units=metric&limit=1&appid=4d9e86afd3e0034d2ad0b63e6c30da28
# forecast weather by lat & long:   api.openweathermap.org/data/2.5/forecast?lat=51.7564&lon=-0.3519&units=metric&limit=1&appid=4d9e86afd3e0034d2ad0b63e6c30da28

# to find lattitude, longitude and city name by postcode and country code:   https://api.openweathermap.org/geo/1.0/zip?zip=AL34TL,GB&appid=4d9e86afd3e0034d2ad0b63e6c30da28
# to find lattitude, longitude and country name by city:    http://api.openweathermap.org/geo/1.0/direct?q=London&limit=5&appid=4d9e86afd3e0034d2ad0b63e6c30da28

import pandas as pd # to use Pandas DataFrames
import requests # to use API
import datetime as datetime # to convert into date & time format. Also, to display current date & time
import os # to check if csv file already exists before exporting the data into csv. Also to delete CSV file 

# Declare variables
location = 'St Albans'
#location = input("Enter city name: ") # asks user to enter city name
full_api_link_current = "https://api.openweathermap.org/data/2.5/weather?q="+location+"&units=metric&limit=1&appid=4d9e86afd3e0034d2ad0b63e6c30da28"
full_api_link_forecast = "https://api.openweathermap.org/data/2.5/forecast?q="+location+"&units=metric&limit=1&appid=4d9e86afd3e0034d2ad0b63e6c30da28"
filename = 'weather.csv'

# Create empty lists to populate with values later on
temp = []
feels_like = []
temp_min = []
temp_max = []
weather_desc = []
wind_speed = []
precipitation = []
city = []
date_time = []
sunrise = []
sunset = []
type = []

def download_API_function(full_api_link): # function to download weather data via API
    api_link = requests.get(full_api_link)
    api_data = api_link.json()
    return api_data

api_data_current = download_API_function(full_api_link_current) # downloads current weather data via API
api_data_forecast = download_API_function(full_api_link_forecast) # downloads forecast weather data via API
#print(api_data_current) # checks if the API request works
#print(api_data_forecast) # checks if the API request works

row_num = len(api_data_forecast['list']) # finds number of rows in the dataset

# Parses current weather data
if api_data_current['cod'] == '404': # if the city doesn't exist, then ask to re-enter correct name
    print("Invalid city: {}, please check you entered it City name correctly".format(location))
else:
    # extracts current weather data and stores it in relevant list variables
    temp.append(api_data_current['main']['temp'])
    feels_like.append(api_data_current['main']['feels_like'])
    temp_min.append(api_data_current['main']['temp_min'])
    temp_max.append(api_data_current['main']['temp_max'])
    weather_desc.append(api_data_current['weather'][0]['description'])
    wind_speed.append(api_data_current['wind']['speed'])
    try: # using try except method to check if 'rain' value is available and captures it in 'precipitation' list. If 'rain' value is not available, then captures '0'
        precipitation.append(api_data_current['rain']['1h'])
    except KeyError:
        precipitation.append(0)
        pass
    city.append(api_data_current['name'])
    epoch_date = (api_data_current['dt'])
    epoch_sunrise = (api_data_current['sys']['sunrise'])
    epoch_sunset = (api_data_current['sys']['sunset'])
    date_time.append(datetime.datetime.fromtimestamp(epoch_date)) # converts epoch to Datetime
    sunrise.append(datetime.datetime.fromtimestamp(epoch_sunrise)) # converts epoch to Datetime
    sunset.append(datetime.datetime.fromtimestamp(epoch_sunset)) # converts epoch to Datetime
    type.append('Current')
    # extracts current weather data and stores it in relevant list variables
    for x in range(row_num):
        temp.append(api_data_forecast['list'][x]['main']['temp'])
        feels_like.append(api_data_forecast['list'][x]['main']['feels_like'])
        temp_min.append(api_data_forecast['list'][x]['main']['temp_min'])
        temp_max.append(api_data_forecast['list'][x]['main']['temp_max'])
        weather_desc.append(api_data_forecast['list'][x]['weather'][0]['description'])
        wind_speed.append(api_data_forecast['list'][x]['wind']['speed'])
        try: # using try except method to check if 'rain' value is available and captures it in 'precipitation' list. If 'rain' value is not available, then captures '0'
            precipitation.append(api_data_forecast['list'][x]['rain']['3h'])
        except KeyError:
            precipitation.append(0)
            pass
        city.append(api_data_forecast['city']['name'])
        epoch_date_val = (api_data_forecast['list'][x]['dt'])
        epoch_sunrise_val = (api_data_forecast['city']['sunrise'])
        epoch_sunset_val = (api_data_forecast['city']['sunset'])
        date_time.append(datetime.datetime.fromtimestamp(epoch_date_val)) # converts epoch to Datetime
        sunrise.append(datetime.datetime.fromtimestamp(epoch_sunrise_val)) # converts epoch to Datetime
        sunset.append(datetime.datetime.fromtimestamp(epoch_sunset_val)) # converts epoch to Datetime
        type.append('Forecast')


# Exports historical and forecast weather data to CSV
df = pd.DataFrame({'city': city, 'temp': temp, 'feels_like': feels_like, 'temp_min': temp_min, 'temp_max': temp_max, 'wind_speed': wind_speed, 'precipitation': precipitation, 'date_time': date_time, 'sunrise': sunrise, 'sunset': sunset, 'type': type}) # creates empty DataFrame and populates it with data from lists
df.to_csv(filename, index=False) # creates CSV file and pastes all data from DataFrame there

In [104]:
import altair as alt # imports altair package

df = pd.melt(df, id_vars=['city', 'type', 'date_time', 'sunrise','sunset'], value_vars=['temp', 'feels_like', 'temp_min', 'temp_max', 'wind_speed', 'precipitation']) # gather columns with numbers into rows and show their results under new 'value' column. This is in order to be able to better chart the data.

alt.Chart(df, title=location).mark_line().encode(
    x=alt.X('date_time:T', axis=alt.Axis(format='%a %H:%M')),
    #x=alt.X('monthdatehours(date_time):T'),
    y='value',
    color='variable'
).interactive().transform_filter(
    {'field': 'variable', 
    'oneOf': ['temp', 'wind_speed', 'precipitation']
    }
    ).properties(
        height=400,
        width=800,
        autosize=alt.AutoSizeParams(
            type='fit',
            contains='padding'
    )) # interactive chart which you can move around and zoom in/out. It shows XY axis line chart, with 'date_time' in X axis in a hours+day+month+year format, 'value' in Y axis for rows where 'value' fields are either 'temp' or 'wind_speed'. Also, resized the chart to increase it.

alt.Chart(...)

In [105]:
# shows time in different timezones (NYC & London)
import pytz # to display current date & time
tz_NY = pytz.timezone('America/New_York') 
datetime_NY = datetime.datetime.now(tz_NY)
NY_time = "NY time:", datetime_NY.strftime("%H:%M")
tz_London = pytz.timezone('Europe/London')
datetime_London = datetime.datetime.now(tz_London)
LN_time = "London time:", datetime_London.strftime("%H:%M")

# pulls data that will later be displayed in DataFrame table
curr_weather = "Weather now", (api_data_current['weather'][0]['description'])
curr_sunrise = "Sunrise", df.at[0, 'sunrise'].strftime("%H:%M")
curr_sunset = "Sunset", df.at[0, 'sunset'].strftime("%H:%M")
curr_date = "Time now", datetime.datetime.now().strftime("%d %b %Y %H:%M")
curr_update_time = "Last updated", df.at[0, 'date_time'].strftime("%H:%M")

# exports data to DataFrame and formats its for better presentation
curr_df = pd.DataFrame({curr_date[0]: curr_date[1:], LN_time[0]: LN_time[1:], NY_time[0]: NY_time[1:], curr_weather[0]: curr_weather[1:], curr_sunrise[0]: curr_sunrise[1:], curr_sunset[0]: curr_sunset[1:], curr_update_time[0]: curr_update_time[1:]}) # creates empty DataFrame and populates it with data from lists
curr_df = curr_df.transpose() # switches rows with columns in DataFrame
curr_df = curr_df.rename(columns={0: "Values"}) # renames '0' column to 'Values'
curr_df.head()

,Values
Time now,19 Nov 2022 23:29
London time:,23:29
NY time:,18:29
Weather now,fog
Sunrise,07:26
